In [3]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3
import pandas as pd
import requests
from datetime import datetime
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [4]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [5]:
session = boto3.Session(profile_name='master')

In [6]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [7]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [8]:
params = {
    'vs_currency' : "usd",
    'days' : "1"
}
res = json.loads(call_get(API_ENDPOINT / "coins" / 'bitcoin' / 'ohlc', params))

In [9]:
df = pd.DataFrame(res, columns=['timestamp', 'open', 'high', 'low', 'close'])
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df['coin'] = 'bitcoin'
df['formated_date'] = df['datetime'].dt.strftime('%Y-%m-%d')

In [10]:
r2 = json.loads(call_get(API_ENDPOINT / "coins" / 'bitcoin' / "market_chart", {"vs_currency" : "usd", "days" : "max"}))

In [107]:
def transform_json_dataframe(data: dict):
    ts = [i[0] for i in data['prices']]
    prices = [i[1] for i in data['prices']]
    market_caps = [i[1] for i in data['market_caps']]
    total_volumes = [i[1] for i in data['total_volumes']]
    temp = pd.DataFrame({
        "timestamp" : ts,
        "prices" : prices,
        "market_caps" : market_caps,
        "total_volumes" : total_volumes,
        "formated_date" : [datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in ts]
        })
    temp['coin'] = 'bitcoin'
    temp['currency'] = 'usd'
    return temp


In [12]:
transform_json_dataframe(r2).dtypes

timestamp          int64
prices           float64
market_caps      float64
total_volumes    float64
formated_date     object
coin              object
currency          object
dtype: object

In [105]:
dynamo = session.resource('dynamodb')
s3_client = session.client('s3')

In [149]:
from boto3.dynamodb.conditions import Key

def convert_lists(response):
    paths = [i['PathS3'] for i in response]
    tokens = [i['TokenName'] for i in response]
    return [list(l) for l in zip(paths, tokens)]


def scan_paths_table(type, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('Pathes')
    scan_kwargs = {
        'FilterExpression': Key('TypeOfRecord').eq(type),
        'ProjectionExpression': "#pth, #tp, #tkn",
        'ExpressionAttributeNames': {"#pth": "PathS3", "#tp": "TypeOfRecord", "#tkn" : "TokenName"}
    }
    results = []
    done = False
    start_key = None
    while not done:
        if start_key:
            scan_kwargs['ExclusiveStartKey'] = start_key
        response = table.scan(**scan_kwargs)
        results.extend(convert_lists(response.get('Items', [])))
        start_key = response.get('LastEvaluatedKey', None)
        done = start_key is None

    return results


In [150]:
a = scan_paths_table('good', dynamo)
a

[['data/raw/bitweb_historical_prices.json', 'bitweb'],
 ['data/raw/babyboo_historical_prices.json', 'babyboo'],
 ['data/raw/bit-ecological-digital_historical_prices.json',
  'bit-ecological-digital'],
 ['data/raw/baby-shark-tank_historical_prices.json', 'baby-shark-tank'],
 ['data/raw/amethyst_historical_prices.json', 'amethyst'],
 ['data/raw/basiscoin-share_historical_prices.json', 'basiscoin-share'],
 ['data/raw/bitau_historical_prices.json', 'bitau'],
 ['data/raw/baby-shark_historical_prices.json', 'baby-shark'],
 ['data/raw/beyond-protocol_historical_prices.json', 'beyond-protocol'],
 ['data/raw/ankr-avalanche-reward-earning-bond_historical_prices.json',
  'ankr-avalanche-reward-earning-bond'],
 ['data/raw/beyondcoin_historical_prices.json', 'beyondcoin'],
 ['data/raw/battle-hero_historical_prices.json', 'battle-hero'],
 ['data/raw/bananaz-club_historical_prices.json', 'bananaz-club'],
 ['data/raw/bit2me_historical_prices.json', 'bit2me'],
 ['data/raw/bela_historical_prices.json', 

In [104]:
def get_json_s3(bucket, key, s3=None):
    """
    Function is used to get JSON file
    from S3 bucket
    """
    if not s3:
        s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [108]:
ddd = get_json_s3('kosmobiker-masterproject', 'data/raw/baby-samo-coin_historical_prices.json', s3_client)
transform_json_dataframe(ddd)

,timestamp,prices,market_caps,total_volumes,formated_date,coin,currency
0,1636675200000,0.001438,0.0,105346.366674,2021-11-12,bitcoin,usd
1,1636761600000,0.001438,0.0,105346.366674,2021-11-13,bitcoin,usd
2,1636848000000,0.002245,0.0,341279.014956,2021-11-14,bitcoin,usd
3,1636934400000,0.001213,0.0,261872.334479,2021-11-15,bitcoin,usd
4,1637020800000,0.001105,0.0,159930.324376,2021-11-16,bitcoin,usd
...,...,...,...,...,...,...,...
129,1647820800000,0.000031,0.0,155.310261,2022-03-21,bitcoin,usd
130,1647907200000,0.000033,0.0,313.921905,2022-03-22,bitcoin,usd
131,1647993600000,0.000039,0.0,1947.112562,2022-03-23,bitcoin,usd
132,1648080000000,0.000040,0.0,339.304013,2022-03-24,bitcoin,usd


In [12]:
def currencies_info(data: dict, list_of_keys: List[str]):
    filtered_json = {key: data[key] for key in list_of_keys}
    temp_df = pd.json_normalize(filtered_json)
    temp_df['timestamp'] = datetime.now().timestamp()


In [93]:
list_of_keys = ['id', 'symbol', 'name', 'hashing_algorithm', 'categories', 'genesis_date',  'sentiment_votes_up_percentage',
                'sentiment_votes_down_percentage', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score',
                'community_score', 'liquidity_score', 'public_interest_score', 'last_updated']

In [94]:
output = {key: res[key] for key in list_of_keys}

In [14]:
session = boto3.Session(profile_name='master')

In [16]:
import awswrangler as wr
wr.catalog.databases(boto3_session=session)

,Database,Description
0,master_project,this is a test database


In [18]:
client = session.client('glue')

In [19]:
client.get_databases()

{'DatabaseList': [{'Name': 'master_project',
   'Description': 'this is a test database',
   'CreateTime': datetime.datetime(2022, 4, 5, 13, 16, 48, tzinfo=tzlocal()),
   'CreateTableDefaultPermissions': [{'Principal': {'DataLakePrincipalIdentifier': 'IAM_ALLOWED_PRINCIPALS'},
     'Permissions': ['ALL']}],
   'CatalogId': '142327076576'}],
 'ResponseMetadata': {'RequestId': '860e955b-1077-406e-8347-1c0b6c746e92',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Apr 2022 11:26:50 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '297',
   'connection': 'keep-alive',
   'x-amzn-requestid': '860e955b-1077-406e-8347-1c0b6c746e92'},
  'RetryAttempts': 0}}